In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
!pip uninstall -y pyarrow requests

!pip install transformers datasets torch evaluate


Found existing installation: pyarrow 14.0.2
Uninstalling pyarrow-14.0.2:
  Successfully uninstalled pyarrow-14.0.2
Found existing installation: requests 2.31.0
Uninstalling requests-2.31.0:
  Successfully uninstalled requests-2.31.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached 

In [ ]:
!pip install accelerate -U
import accelerate
print(accelerate.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.1 MB/s eta 0:00:00
0.32.1


In [ ]:
!pip uninstall transformers datasets torch
!pip cache purge

In [ ]:
!pip install evaluate

In [ ]:
import accelerate
print(accelerate.__version__)

ModuleNotFoundError: No module named 'accelerate'

In [ ]:
from datasets import load_dataset, DatasetDict
import pandas as pd

# Load your XGLUE JavaScript dataset
dataset = load_dataset('code_x_glue_ct_code_to_text', 'javascript')

# Convert the dataset to Pandas DataFrames
train_df = pd.DataFrame(dataset['train'])
val_df = pd.DataFrame(dataset['validation'])
test_df = pd.DataFrame(dataset['test'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/58025 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3885 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3291 [00:00<?, ? examples/s]

In [ ]:
!pip install datasets sacrebleu rouge_score nltk

In [ ]:
import evaluate
from datasets import load_metric

# Load metrics
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Calculate BLEU
    bleu = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Calculate ROUGE
    rouge = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Calculate METEOR
    meteor = meteor_metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Return combined metrics
    return {
        "bleu": bleu["bleu"],
        "rouge1": rouge["rouge1"].mid.fmeasure,
        "rouge2": rouge["rouge2"].mid.fmeasure,
        "rougeL": rouge["rougeL"].mid.fmeasure,
        "meteor": meteor["meteor"]
    }


[nltk_data] Downloading package wordnet to /home/nm788186/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/nm788186/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/nm788186/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
pip install rouge_score

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip freeze

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=5aedd0e656c31264fa6cce88802326c8b242051825f02b64ddb9aee07c432952
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip install sentencepiece


In [ ]:
save_path = '/content/drive/MyDrive/'

# Save the DataFrames to CSV files
train_df.to_csv(f'{save_path}train1.csv', index=False)
val_df.to_csv(f'{save_path}val1.csv', index=False)
test_df.to_csv(f'{save_path}test1.csv', index=False)

In [ ]:
import pandas as pd
import torch
from datasets import Dataset, DatasetDict, load_metric
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer

# Initialize the tokenizer
model_name = "t5-small"  # Change to your model name if different
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Ensure the model path exists
model_path = '/data/T5/enhanced_model5_53k'
try:
    model = T5ForConditionalGeneration.from_pretrained(model_path)
except Exception as e:
    print(f"Error loading model: {e}")

# Filter datasets based on code_tokens_length
def filter_and_sample_df(df, column_name, max_length, n_samples):
    df['code_tokens_length'] = df[column_name].apply(lambda x: len(tokenizer.encode(x)))
    sampled_df = df[df['code_tokens_length'] <= max_length]
    return sampled_df.sample(n=min(n_samples, len(sampled_df)), random_state=123)

# Load and filter datasets (replace with actual data loading)
# train_df, val_df, test_df should be your actual data
# train_df, val_df, test_df = load_your_data()

sampled_train_df = filter_and_sample_df(train_df, 'code', 512, 53000)
sampled_val_df = filter_and_sample_df(val_df, 'code', 512, 3000)
sampled_test_df = filter_and_sample_df(test_df, 'code', 512, 3000)

# Convert DataFrames to Hugging Face Dataset
def convert_to_hf_dataset(df):
    return Dataset.from_pandas(df)

train_data = convert_to_hf_dataset(sampled_train_df)
val_data = convert_to_hf_dataset(sampled_val_df)
test_data = convert_to_hf_dataset(sampled_test_df)

filtered_dataset = DatasetDict({
    'train': train_data,
    'validation': val_data,
    'test': test_data
})

# Tokenize the filtered dataset
def tokenize_function(examples):
    model_inputs = tokenizer(examples['code'], max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(examples['docstring'], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = filtered_dataset.map(tokenize_function, batched=True)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load metric
rouge = load_metric("rouge")

# Function to compute metrics


# Training arguments
training_args = TrainingArguments(
    output_dir='/data/T5/results53k_4',
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    weight_decay=0.01,
    lr_scheduler_type='linear',
    warmup_steps=500,
    logging_dir='/data/logs53k_4',
    logging_steps=10,
    gradient_accumulation_steps=2,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained('/data/T5/enhanced_model6_53k')
tokenizer.save_pretrained('/data/T5/enhanced_model6_53k')

# Evaluate the model
results = trainer.evaluate()
print(results)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Token indices sequence length is longer than the specified maximum sequence length for this model (984 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/53000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, RobertaTokenizer

# Load the trained model and tokenizer
model_path = '/content/drive/MyDrive/codeT5_20k'
tokenizer =  RobertaTokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

def generate_summary(code_snippet):
    # Tokenize the input code snippet
    inputs = tokenizer(code_snippet, return_tensors='pt', max_length=512, truncation=True, padding="max_length")

    # Generate the summary
    summary_ids = model.generate(inputs['input_ids'], max_length=128, num_beams=4, early_stopping=True)

    # Decode the generated tokens into a readable string
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

# Example usage
code_snippet = """

let numbers = [1, 2, 3, 4, 5];
let doubled = numbers.map(num => num * 2);

console.log(doubled);

"""
summary = generate_summary(code_snippet)
print(summary)


Calculates the double-precision of a number.

@param {number[]} numbers The numbers.
@param {number} 2 The 2.
@param {number} 3 The 3.
@param {number} 4 The 4.
@param {number} 5 The 5.


In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd
from datasets import Dataset, load_metric
import evaluate

# Load your fine-tuned model and tokenizer
model_path = '/data/T5/enhanced_model5_53k'  # Path where your model is saved
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load your test dataset (replace this with actual loading code)
# test_df = pd.read_csv('path/to/your/test_data.csv')

# Sample 10 examples from the test dataset
sampled_test_df = test_df.sample(n=100, random_state=123)

# Convert to Hugging Face Dataset
sampled_test_data = Dataset.from_pandas(sampled_test_df)

# Generate predictions for the selected samples
def generate_predictions(model, tokenizer, dataset, device):
    model.eval()
    predictions = []
    references = []

    for example in dataset:
        input_text = example['code']
        reference_text = example['docstring']

        input_ids = tokenizer.encode(input_text, padding="max_length", truncation=True, return_tensors='pt').to(device)

        with torch.no_grad():
            outputs = model.generate(input_ids, max_length=512, num_beams=16, early_stopping=True, length_penalty=6)

        predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(predicted_text)
        references.append(reference_text)

    return predictions, references

# Generate predictions
predictions, references = generate_predictions(model, tokenizer, sampled_test_data, device)

# Load metrics
rouge_metric = load_metric("rouge")
meteor_metric = evaluate.load("meteor")

# Compute ROUGE scores
rouge_results = rouge_metric.compute(predictions=predictions, references=references)

# Compute METEOR scores
meteor_results = meteor_metric.compute(predictions=predictions, references=references)

# Print results
print("ROUGE-1:", rouge_results['rouge1'].mid.fmeasure)
print("ROUGE-2:", rouge_results['rouge2'].mid.fmeasure)
print("ROUGE-L:", rouge_results['rougeL'].mid.fmeasure)
print("METEOR:", meteor_results['meteor'])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/tmp/ipykernel_3616748/2371260524.py:50: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")
[nltk_data] Downloading package wordnet to /home/nm788186/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/nm788186/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/nm788186/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


ROUGE-1: 0.3005152640590225
ROUGE-2: 0.10932512927612076
ROUGE-L: 0.26384747255403634
METEOR: 0.20836361383375915


In [ ]:
print(f"reference: {references[56]}\n")
print(f'prediction: {predictions[56]}')


reference: Overrides FeatureGroup.onAdd

prediction: Adds markers to the map @param Object map Map of markers


In [ ]:

#evaluation of entire test data with the summaries generated
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

import evaluate
from datasets import Dataset

# Determine the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the saved model and tokenizer
model_path = '/content/drive/MyDrive/enhanced_model2_53k'
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)
model.to(device)

# Function to generate a summary from a code snippet
def generate_summary(code_snippet, model, tokenizer):
    input_text = f"code to summarize: {code_snippet}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    inputs = inputs.to(device)

    summary_ids = model.generate(
        inputs.input_ids,
        max_length=128,
        min_length=40,
        length_penalty=2.0,
        num_beams=5,
        early_stopping=True,
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary




# Convert DataFrames back to Hugging Face Dataset
test_data = Dataset.from_pandas(test_df)

# Generate summaries for the test dataset
generated_summaries = []
for code_snippet in test_data['code']:
    summary = generate_summary(code_snippet, model, tokenizer)
    generated_summaries.append(summary)

# Function to evaluate the generated summaries
def evaluate_summaries(references, predictions):
    # Initialize the metrics

    rouge = evaluate.load("rouge")
    meteor = evaluate.load("meteor")

    # Calculate the metrics

    rouge_score = rouge.compute(predictions=predictions, references=references)
    meteor_score = meteor.compute(predictions=predictions, references=references)

    return {

        "rouge": rouge_score,
        "meteor": meteor_score
    }

# Evaluate the generated summaries against the reference summaries
metrics = evaluate_summaries(test_data['docstring'], generated_summaries)

print("Generated Summaries:")
for gs in generated_summaries:
    print(gs)

print("Evaluation Metrics:")
print(metrics)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the trained model and tokenizer
model_path = '/content/drive/MyDrive/enhanced_model2_53k'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to generate summaries
def generate_summary(example):
    inputs = tokenizer(example['code'], return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    summary_ids = model.generate(inputs['input_ids'], max_length=128, num_beams=5, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Example usage
test_samples = ['''
    def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr
    '''
]

# Generate summaries for the example test samples
generated_summaries = [generate_summary(sample) for sample in test_samples]
print(generated_summaries)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


TypeError: string indices must be integers